
# IEEE - CIS Fraud Detection


**들어가기에 앞서** <br>
온라인 상에서의 사기 거래는 매년 수십억 달러에 달하며, 계속 증가하고 있다. 기업들은 이를 막기 위하여 사기 예방 시스템을 도입하여 사기 거래를 방지하고자 하고 있다. <br>IEEE-CIS는 deep neural networks, fuzzy systems, evolutionary computation과 swarm intelligence 등 다양한 인공지능 및 기계학습을 통해 이를 처리하고 있다. <br>이 대회에서는 Vesta에서 제공한 데이터를 가지고 예측을 수행하게 되는데, 제공되는 데이터는 실제 거래 내역을 바탕으로 하여 마스킹 된 부분이 많다.


이 대회는 **binary classification** 문제로, 온라인 거래가 사기일 확률을 예측하는 것이 목적이다.<br>
우리는 fraud client가 되어버린 user계정이 계속하여 사기에 이용될 가능성을 발견하고, 이를 바탕으로 온라인 사기 확률을 예측하고자 한다.



### 참고자료 :

- https://www.kaggle.com/artgor/eda-and-models/data
- https://www.kaggle.com/artkulak/ieee-fraud-simple-baseline-0-9383-lb
- https://www.kaggle.com/robikscube/ieee-fraud-detection-first-look-and-eda
- https://www.kaggle.com/mjbahmani/reducing-memory-size-for-ieee
- https://www.kaggle.com/jesucristo/fraud-complete-eda


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

import matplotlib.gridspec as gridspec
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import gc
gc.enable()

import os
print(os.listdir("../input"))

# 1. 데이터 탐색

## 데이터 개관

주 데이터는  **identity** 와 **transaction**으로 나뉘며, ```TransactionID```을 공통분모로 가지고 있다.



**Transaction Table**
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp) *-- 지정된 참조 날짜/시간(실제 타임스탬프가 아님)의 시간 초과--*
- TransactionAMT: transaction payment amount in USD *--트랜잭션 결제 금액(USD)--*
- ProductCD: product code, the product for each transaction*--제품 코드, 각 거래에 대한 제품--*
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.*--카드 종류, 카드 카테고리, 발행 은행, 국가 등 결제 '카드' 정보--*
- addr: 주소
- dist: 거리
- P_ and (R__) emaildomain: purchaser and recipient email domain *--구매자 및 수신자 이메일 도메인--*
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked. *--결제 카드와 관련된 주소의 수와 같은 수. (실제 의미는 마스킹처리)--*
- D1-D15: timedelta, such as days between previous transaction, etc. *--이전 거래 사이의 일 등 시간 지정--*
- M1-M9: match, such as names on card and address, etc. *--카드의 이름, 주소등이 일치하는지 여부--*
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations. *--Vest는 순위, 계수 및 기타 엔티티 관계를 포함하여 풍부한 기능을 설계했다.-*

**Categorical Features:**
- ProductCD
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

**Identity Table**
- Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.<br> *--이 표의 변수는 트랜잭션과 관련된 ID 정보 : 네트워크 연결 정보(IP, ISP, 프록시 등) 및 디지털 서명(UA/브라우저/OS/버전 등)--*

**Categorical Features:**
- DeviceType
- DeviceInfo
- id_12 - id_38

**Files**

- train_{transaction, identity}.csv - the training set
- test_{transaction, identity}.csv - the test set
- sample_submission.csv


## 데이터 불러오기

In [ ]:
train_transaction = pd.read_csv('../input/train_transaction.csv')
test_transaction = pd.read_csv('../input/test_transaction.csv')
train_identity = pd.read_csv('../input/train_identity.csv')
test_identity = pd.read_csv('../input/test_identity.csv')
print ("Data is loaded!")

In [ ]:
print('train_transaction shape is {}'.format(train_transaction.shape))
print('test_transaction shape is {}'.format(test_transaction.shape))
print('train_identity shape is {}'.format(train_identity.shape))
print('test_identity shape is {}'.format(test_identity.shape))

In [ ]:
train_transaction.info()
train_transaction.describe()
train_transaction.head()

In [ ]:
train_identity.info()
train_identity.describe()
train_identity.head()

#### **train_transaction**

In [ ]:
missing_values_count = train_transaction.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(train_transaction.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

#### **train_identity**

In [ ]:
missing_values_count = train_identity.isnull().sum()
print (missing_values_count[0:10])
total_cells = np.product(train_identity.shape)
total_missing = missing_values_count.sum()
print ("% of missing data = ",(total_missing/total_cells) * 100)

In [ ]:
# 'TransactionID'에 대해 유니크한 속성 개수 & 비율
print(np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique())))
print(np.sum(test_transaction['TransactionID'].isin(test_identity['TransactionID'].unique())))
print()
print(round(144233 / 590540 *100, 1), "% => 144233 / 590540 --TransactionIDs in train, have an associated train_identity")
print(round(141907 / 506692 *100, 1), "% => 141907 / 506692 --TransactionIDs in test, have an associated test_identity")

메모리 확보를 위해 필요 없는 변수는 바로 정리한다.

In [ ]:
del missing_values_count, total_cells, total_missing

### 데이터 불균형 문제 + @


In [ ]:
train_transaction.groupby('isFraud').count()['TransactionID'].plot(kind = 'barh',
                                                                   title = 'Distribution of Target(isFraud) in Train',
                                                                   figsize = (20, 10),
                                                                   fontsize=20)

plt.show()

#### 위 표에서 볼 수 있듯이 사기 거래인 데이터는 전체의 약 3.5%에 못 미치며 데이터 불균형을 초래하고 있다.
#### 또한 데이터에 대한 이슈는 아래에서 계속 찾아볼 수 있는데, Train data와 Test data 사이 시간적 공백이 존재한다는 것이다.

In [ ]:
print('     {:.4f} % of Transactions that are fraud in train.'.format(train_transaction['isFraud'].mean() * 100))

In [ ]:
train_transaction['TransactionDT'].plot(kind='hist', figsize=(15,5), bins=100, label='Train', title='Train vs. Test TransactionDT distribution')
test_transaction['TransactionDT'].plot(kind='hist', label='test', bins=100)
plt.legend()
plt.show()

In [ ]:
# 전체 분포
ax = train_transaction.plot(x = 'TransactionDT' ,
                            y = 'TransactionAmt',
                            kind = 'scatter',
                            alpha = 0.2,
                            label = 'Train adn Test Transaction Amounts by Time(TransactionDT)',
                            ylim = (0, 5000),
                            figsize=(15,10)
                            ) ;

# test 트랜잭션 분포
test_transaction.plot(x = 'TransactionDT',
                      y = 'TransactionAmt',
                      kind = 'scatter',
                      label = 'TransactionAmt - test',
                      alpha = 0.2,
                      color = 'green',
                      ylim = (0, 5000),
                      ax = ax) ;

# 학습 트랜잭션 중 사기인 것들 분포
# Plot Fraud as Orange
train_transaction.loc[train_transaction['isFraud'] == 1].plot(
                                                                x = 'TransactionDT',
                                                                y = 'TransactionAmt',
                                                                kind = 'scatter',
                                                                alpha = 0.2,
                                                                label = 'TransactionAmt - train',
                                                                title = 'Train and Test Transaction Amounts by Time (TransactionDT)',
                                                                ylim = (0, 5000),
                                                                color = 'orange',
                                                                figsize = (15, 10),
                                                                ax = ax
                                                             ) ;

plt.show()

### Train과 Test 사이 공백을 구해보자.
> **The TransactionDT feature is a timedelta from a given reference datetime (not an actual timestamp).**<br>
> TransactionDT feature는 주어진 날짜(시간)간의 연산이 가능한 timedelta.

**[The timespan of the dataset is 1 year ?[The timespan of the dataset is 1 year ?][The timespan of the dataset is 1 year ?[The timespan of the dataset is 1 year ?] <br>
(https://www.kaggle.com/c/ieee-fraud-detection/discussion/100071#latest-577632) by Suchith** <br>를 참고하여 이 공백이 약 30일, 한달 정도를 의미한다는 것을 알 수 있었다.


```
Train: min = 86400 max = 15811131
Test: min = 18403224 max = 34214345
```

The difference train.min() and test.max() is ```x = 34214345 - 86400 = 34127945``` but we don't know is it in seconds,minutes or hours.

```
Time span of the total dataset is 394.9993634259259 days
Time span of Train dataset is  181.99920138888888 days
Time span of Test dataset is  182.99908564814814 days
The gap between train and test is 30.00107638888889 days
```

If it is in seconds then dataset timespan will be ```x/(3600*24*365) = 1.0821``` years which seems reasonable to me. So if the **transactionDT** is in **seconds** then

```
Time span of the total dataset is 394.9993634259259 days
Time span of Train dataset is  181.99920138888888 days
Time span of Test dataset is  182.99908564814814 days
The gap between train and test is 30.00107638888889 days
```


![](https://www.googleapis.com/download/storage/v1/b/kaggle-user-content/o/inbox%2F2370491%2Fc9bf5af5e902595b737df5470adc193b%2Fdownload-1.png?generation=1563312982845419&alt=media)

**source: [FChmiel](https://www.kaggle.com/fchmiel)**
<br>

#### 우리는 위에서 TransactionID가 각 train과 test data에서 차지하는 비율을 확인했다.

cf)
```24.4%``` of TransactionIDs in train (144233 / 590540) have an associated train_identity.<br>
```28.0%``` of TransactionIDs in test (144233 / 590540) have an associated train_identity.

#### 이를 바탕으로 TransactionDT에 대해 더 알아보고자 한다.

In [ ]:
train_transaction['TransactionDT'].head()

In [ ]:
train_transaction['TransactionDT'].shape[0] , train_transaction['TransactionDT'].nunique()

In [ ]:
train_transaction['TransactionDT'].value_counts().head(10)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

time_val = train_transaction['TransactionDT'].values

sns.distplot(time_val, ax=ax[0], color='r')
ax[0].set_title('Distribution of TransactionDT', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Distribution of LOG TransactionDT', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

plt.show()

#### 더 자세히 들여다 보기 위해 0과 1 사이에 있는 value를 음수로 만들기 위해 log를 씌우고, TransactionDT의 최소부터 최대까지를 범위로 하는 그래프를 그렸다.

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

time_val = train_transaction.loc[train_transaction['isFraud'] == 1]['TransactionDT'].values

sns.distplot(np.log(time_val), ax=ax[0], color='r')
ax[0].set_title('Distribution of LOG TransactionDT, isFraud=1', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])

time_val = train_transaction.loc[train_transaction['isFraud'] == 0]['TransactionDT'].values

sns.distplot(np.log(time_val), ax=ax[1], color='b')
ax[1].set_title('Distribution of LOG TransactionDT, isFraud=0', fontsize=14)
ax[1].set_xlim([min(np.log(time_val)), max(np.log(time_val))])


plt.show()

#### 사기 거래의 경우, 사기가 이루어진 어느 시점의 이후 더이상 접속하지 않는다는 것을 알 수 있었다.

### TransactionAmt

In [ ]:
train_transaction['TransactionAmt'] \
    .apply(np.log) \
    .plot(kind = 'hist',
          bins = 100,
          figsize=(15, 10),
          title = 'Distribution of Log Transaction Amt') ;

plt.show()

In [ ]:

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10)) ;

train_transaction.loc[train_transaction['isFraud'] == 1] ['TransactionAmt'].apply(np.log) \
    .plot(kind = 'hist',
          bins = 100,
          title = 'Log Transaction Amt = Fraud',
          color = color_pal[1],
          xlim = (-3, 10),
          ax = ax1)
    
train_transaction.loc[train_transaction['isFraud'] == 0] ['TransactionAmt'].apply(np.log) \
    .plot(kind = 'hist',
          bins = 100,
          title = 'Log Transaction Amt = Not Fraud',
          color = color_pal[2],
          xlim = (-3, 10),
          ax = ax2)
    
train_transaction.loc[train_transaction['isFraud'] == 1] ['TransactionAmt'] \
    .plot(kind = 'hist',
          bins = 100,
          title = 'Transaction Amt = Fraud',
          color = color_pal[1],
          ax = ax3)
    
train_transaction.loc[train_transaction['isFraud'] == 0] ['TransactionAmt'] \
    .plot(kind = 'hist',
          bins = 100,
          title = 'Transaction Amt = Not Fraud',
          color = color_pal[2],
          ax = ax4)

#### 사기인 거래의 금액과 아닌 거래의 금액의 전체 분포는 비슷한 양상을 띄지만, 사기 금액이 전체 거래 금액에 비해 높은 것으로 나타난다.

In [ ]:
print('Mean transaction amt for fraud is {:.4f}' \
      .format(train_transaction.loc[train_transaction['isFraud'] == 1] ['TransactionAmt'] \
              .mean())) 

print('Mean transaction amt for non-fraud is {:.4f}' \
      .format(train_transaction.loc[train_transaction['isFraud'] == 0] ['TransactionAmt'] \
              .mean())) 

### ProductCD

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

train_transaction.groupby('ProductCD')['TransactionID'] \
    .count() \
    .sort_index() \
    .plot(kind = 'barh',
          title = 'Count of Observations by ProductCD',
          fontsize=20,
          ax = ax1)


train_transaction.groupby('ProductCD')['isFraud'] \
    .mean() \
    .sort_index() \
    .plot(kind='barh',
          title = 'Percentage of Fraud by ProductCD',
          fontsize=20,
          ax=ax2)
    
plt.show() ;

### 제품코드에 대해 알 수 있는 것들
#### 각각이 어떤 제품을 대표하는 제품코드인지는 알 수 없으나, 
- ```W``` 가 제일 많은 수로 구매한 제품 코드이고, ```S``` 가 제일 적은 수로 구매한 제품코드이다.
- productCD ```C``` 는 동 제품코드 전체의 11% 이상에서 사기 거래 비율을 차지한다.
- productCD ```W``` 는 동 제품코드 전체의 2% 이하의 사기 거래 비율이 나타난다.

### C1 ~ C14 (결제카드와 같은 주소의 수)

In [ ]:
c_cols = [c for c in train_transaction if c[0] == 'C']
train_transaction[c_cols].describe()

In [ ]:
train_transaction.loc[train_transaction['isFraud'] == 1]
train_transaction.loc[train_transaction['isFraud'] == 0]

In [ ]:
# Sample 500 Fraud adn 500 Non Fraud examples to plot
sampled_train = pd.concat(
                         [
                         train_transaction.loc[train_transaction['isFraud'] == 0 ].sample(500),
                         train_transaction.loc[train_transaction['isFraud'] == 1].sample(500)
                         ]
                        )

sampled_train

### D1 ~ D9 : ```D``` feature--이전 거래 사이의 일 등 시간 지정--의 대부분이 NaN 값인 것을 확인할 수 있다.

In [ ]:
d_cols = [d for d in train_transaction if d[0] == 'D']
train_transaction[d_cols].head()

In [ ]:
# D feature의 결측치 개수
missing_values_count = train_transaction[d_cols].isnull().sum()
missing_values_count

In [ ]:
total_cells = np.product(train_transaction[d_cols].shape)
total_missing = missing_values_count.sum()

(total_missing/total_cells) * 100

### M1 ~ M9

In [ ]:
m_cols = [m for m in train_transaction if m[0] == 'M']
train_transaction[m_cols].head()

In [ ]:
(train_transaction[m_cols] == 'T').sum().plot(kind='bar', title='Count of T by M column', figsize = (20, 5), color=color_pal[3])
plt.show()

(train_transaction[m_cols] == 'F').sum().plot(kind='bar', title='Count of F by M column', figsize = (20, 5), color=color_pal[4])
plt.show()

(train_transaction[m_cols].isna()).sum().plot(kind='bar', title='Count of Nan by M column', figsize = (20, 5), color=color_pal[5])
plt.show()

- M의 속성들은 T, F 아니면 결측치이다.
- M4는 결측치만 존재한다.

### V1 ~ V339

In [ ]:
v_cols = [v for v in train_transaction if v[0] == 'V']
train_transaction[v_cols].head()

In [ ]:
train_transaction[v_cols].describe()

In [ ]:
train_transaction[v_cols].mean(axis=0) # 로우를 축으로 한 평균

In [ ]:
train_transaction[v_cols].mean(axis=1) # 컬럼을 축으로 한 평균 --index 별 평균

In [ ]:
train_transaction['v_mean'] = train_transaction[v_cols].mean(axis=1)

fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(20, 10)) # sharex : 복수 그래프 간 x 축을 공유
train_transaction.loc[train_transaction['isFraud'] == 1]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log transformed mean of V columns - Fraud',
          ax=ax1)
    
train_transaction.loc[train_transaction['isFraud'] == 0]['v_mean'] \
    .apply(np.log) \
    .plot(kind='hist',
          bins=100,
          title='Log transformed mean of V columns - Non Fraud',
          color=color_pal[1],
          ax=ax2)
    
plt.show()

# Unique Values

### D Features

In [ ]:
plt.figure(figsize=(10, 7))
d_features = list(train_transaction.columns[31:46])
uniques = [len(train_transaction[col].unique()) for col in d_features]
sns.set(font_scale=1.2)
ax = sns.barplot(d_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(10, 7))
d_features = list(test_transaction.columns[31:46])
uniques = [len(test_transaction[col].unique()) for col in d_features]
sns.set(font_scale=1.2)
ax = sns.barplot(d_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

### C features

In [ ]:
plt.figure(figsize=(10, 7))
c_features = list(train_transaction.columns[17:31])
uniques = [len(train_transaction[col].unique()) for col in c_features]
sns.set(font_scale=1.2)
ax = sns.barplot(c_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(10, 7))
c_features = list(test_transaction.columns[17:31])
uniques = [len(test_transaction[col].unique()) for col in c_features]
sns.set(font_scale=1.2)
ax = sns.barplot(c_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

### V features

In [ ]:
plt.figure(figsize=(35, 8))
v_features = list(train_transaction.columns[55:121])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(35, 8))
v_features = list(train_transaction.columns[121:171])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(35, 8))
v_features = list(train_transaction.columns[171:221])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(35, 8))
v_features = list(train_transaction.columns[221:271])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(35, 8))
v_features = list(train_transaction.columns[271:321])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(38, 8))
v_features = list(train_transaction.columns[321:391])
uniques = [len(train_transaction[col].unique()) for col in v_features]
sns.set(font_scale=1.2)
ax = sns.barplot(v_features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

### id_code

In [ ]:
train_identity.head(2)

In [ ]:
plt.figure(figsize=(35, 8))
features = list(train_identity.columns[1:39])
uniques = [len(train_identity[col].unique()) for col in features]
sns.set(font_scale=1.2)
ax = sns.barplot(features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TRAIN')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

In [ ]:
plt.figure(figsize=(35, 8))
features = list(test_identity.columns[1:39])
uniques = [len(test_identity[col].unique()) for col in features]
sns.set(font_scale=1.2)
ax = sns.barplot(features, uniques, log=True)
ax.set(xlabel='Feature', ylabel='log(unique count)', title='Number of unique values per feature TEST')
for p, uniq in zip(ax.patches, uniques):
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 10,
            uniq,
            ha="center") 

### Device Type & Device Info

In [ ]:
ax = sns.countplot(x="DeviceType", data=train_identity)
ax.set_title('DeviceType', fontsize=14)
plt.show()

In [ ]:
print ("Unique Devices = ",train_identity['DeviceInfo'].nunique())
train_identity['DeviceInfo'].value_counts().head()

### Card

In [ ]:
cards = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6']
for i in cards:
    print ("Unique ",i, " = ",train_transaction[i].nunique())

In [ ]:
print('addr1 : has {} NA values'.format(train_transaction['addr1'].isna().sum()))
print('addr2 : has {} NA values'.format(train_transaction['addr2'].isna().sum()))
print('addr1 has ', train_transaction['addr1'].count(), 'attributes, and addr2 has', train_transaction['addr2'].count(), 'attributes')
print('NA Percentage : ', round(524834 / 65706, 2), '%')

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(25,5))

sns.countplot(x="card4", ax=ax[0], data=train_transaction.loc[train_transaction['isFraud'] == 0])
ax[0].set_title('card4 isFraud=0', fontsize=14)
sns.countplot(x="card4", ax=ax[1], data=train_transaction.loc[train_transaction['isFraud'] == 1])
ax[1].set_title('card4 isFraud=1', fontsize=14)
sns.countplot(x="card6", ax=ax[2], data=train_transaction.loc[train_transaction['isFraud'] == 0])
ax[2].set_title('card6 isFraud=0', fontsize=14)
sns.countplot(x="card6", ax=ax[3], data=train_transaction.loc[train_transaction['isFraud'] == 1])
ax[3].set_title('card6 isFraud=1', fontsize=14)
plt.show()

- Visa 카드가 사용률이 제일 높고 카드에 따른 사기/정상 비율에서는 특이점이 없다.
- 체크카드보다 신용카드가 사기 당하지 않은 수가 더 많고, 사기 당한 카드 중에서는 신용카드보다 체크카드가 더 많다.

In [ ]:
cards = train_transaction.iloc[:,5:8].columns

plt.figure(figsize=(18,8*4))
gs = gridspec.GridSpec(8, 4)
for i, cn in enumerate(cards):
    ax = plt.subplot(gs[i])
    sns.distplot(train_transaction.loc[train_transaction['isFraud'] == 1][cn], bins=50)
    sns.distplot(train_transaction.loc[train_transaction['isFraud'] == 0][cn], bins=50)
    ax.set_xlabel('')
    ax.set_title('feature: ' + str(cn))
plt.show()


### Email Domain

In [ ]:
"emaildomain" in train_transaction.columns, "emaildomain" in train_identity.columns

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(32,10))

sns.countplot(y="P_emaildomain", ax=ax[0], data=train_transaction)
ax[0].set_title('P_emaildomain', fontsize=14)
sns.countplot(y="P_emaildomain", ax=ax[1], data=train_transaction.loc[train_transaction['isFraud'] == 1])
ax[1].set_title('P_emaildomain isFraud = 1', fontsize=14)
sns.countplot(y="P_emaildomain", ax=ax[2], data=train_transaction.loc[train_transaction['isFraud'] == 0])
ax[2].set_title('P_emaildomain isFraud = 0', fontsize=14)
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(32,10))

sns.countplot(y="R_emaildomain", ax=ax[0], data=train_transaction)
ax[0].set_title('R_emaildomain', fontsize=14)
sns.countplot(y="R_emaildomain", ax=ax[1], data=train_transaction.loc[train_transaction['isFraud'] == 1])
ax[1].set_title('R_emaildomain isFraud = 1', fontsize=14)
sns.countplot(y="R_emaildomain", ax=ax[2], data=train_transaction.loc[train_transaction['isFraud'] == 0])
ax[2].set_title('R_emaildomain isFraud = 0', fontsize=14)
plt.show()

#### gmail을 domain으로 하는 사기 거래가 압도적이었다.
#### 그러나 사기 거래가 아닌 정상 거래의 domain도 gmail domain이 최다를 차지하여 정보로써 활용도가 떨어진다.

### 결론

#### EDA 결과 우리는 사기거래가 일어나는 것을 찾는게 아닌 사기가 발생하여 fraud client가 되버린 user계정을 찾는데 집중해야한다고 결론지었다.
- 한 번 이상 사기 거래에 이용된 fraud user id에서는 계속 사기 거래가 이루어지거나, 사기 거래 이후 아예 이용되지 않는다.
- 따라서 fraud user id를 알아내면 해당 계정에서 일어나는 모든 transaction을 사기라고 분류해낼 수 있을 것이다.
- fraud user id를 찾아내는 데에 D, V, ID column들과 같은 특정 user를 식별하는데 도움을 주는 열을 그대로 사용하면 과적합이 일어날수 있으므로,
- 기존의 user id를 사용하는 대신 새로운 변수를 만들어 사용해야 model이 unseen user id에 대해 과적합 없이 분류할 수 있을 것이다.